In [6]:
import torch
from matplotlib import pyplot as plt 
import numpy as np
import random

num_inputs = 2
num_features = 1000
true_w = [2,-3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0,1,(num_features,num_inputs)),dtype=torch.float32)
labels = true_w[0] * features[:,0] + true_w[1] * features[:,1] + true_b
labels += torch.tensor(np.random.normal(0,0.01,size=labels.size()),dtype=torch.float32)

print(features[0],labels[0])

tensor([ 0.5646, -0.1542]) tensor(5.8542)


## 使用Dataloader读取数据
在前面中，我们自定义了`data_iter`来遍历训练样本，在Pytorch中封装了`torch.utils.data`包进行数据的读取与处理。对于简单的数据集来说，可以直接使用`TensorDataset`包装，然后使用`Dataloader`进行读取。 对于复杂或者无法一次加载到内存中的数据计，可以自定义`Dataset`。

In [7]:
import torch.utils.data as Data 

batch_size = 10
dataset = Data.TensorDataset(features,labels)
data_iter = Data.DataLoader(dataset,batch_size=batch_size,shuffle=True)

for X,y in data_iter:
    print(X,y)
    break

tensor([[ 0.7919,  0.8870],
        [ 1.7547, -0.4990],
        [ 1.2296,  1.9101],
        [ 0.9744, -2.0938],
        [ 1.2899, -0.0490],
        [ 0.5929, -1.6408],
        [-1.7833,  0.3483],
        [-1.7012,  0.4776],
        [-2.0966,  0.3745],
        [-1.7099,  0.8016]]) tensor([ 2.7933,  9.4112,  0.1571, 13.2538,  6.9473, 10.9624, -0.5440, -0.8362,
        -1.2595, -1.9642])


## 定义网络模型

这里自定义一个网络层来实现线性回归。 首先，导入`torch.nn`模块，nn表示**neural networks**,该模块定义了大量神经网络的层。`nn`的核心数据结构为`Module`，是一个抽象的概念，**即可以表示神经网络中的某个层（layer），也可以表示一个包含很多层的的网络。

在实际使用中，通常可以继承`nn.Module`，实现自定义的网络/层。一个`nn.Module`实例应该包含一些层以及返回输出结果的前向传播(forward)方法。


In [9]:

import torch.nn as nn
class LinearNet(nn.Module):
    def __init__(self,n_feature):
        super(LinearNet,self).__init__()
        self.linear = nn.Linear(n_feature,1)

    def forward(self,x):
        y = self.linear(x)
        return y

net = LinearNet(num_inputs)
print(net)

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


可以使用PyTorch提供的容器`Sequential`来方便快捷的搭建网络。`Sequential`是一个有序容器，网络层将按照加入到`Sequential`的顺序被依次添加到计算图中。
```
# 写法一
net = nn.Sequential(
    nn.Linear(num_inputs,1)
)
# 写法二
net = nn.Sequential()
net.add_module('linear',nn.Linear(num_inputs,1))
# net.add_module ...

# 写法三
from collection import OrdereDict
net = nn.Sequential(OrdereDict([
    ('linear',nn.Linear(num_inputs,1))
    # ()...
    ]))
```